In [14]:
pip install imblearn

   ---------------------------------------- 0.0/238.4 kB ? eta -:--:--
   ---------------------------------------  235.5/238.4 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 238.4/238.4 kB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   ---------------------------------------- 301.8/301.8 kB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 11.1 MB/s eta 0:00:01
   ---- ----------------------------------- 1.3/11.1 MB 14.1 MB/s eta 0:00:01
   ------- -------------------------------- 2.1/11.1 MB 15.1 MB/s eta 0:00:01
   ---------- ----------------------------- 2.8/11.1 MB 14.9 MB/s eta 0:00:01
   ------------ --------------------------- 3.6/11.1 MB 15.2 MB/s eta 0:00:01
   --------------- ------------------------ 4.3/11.1 MB 14.3 MB/s eta 0:00:01
   ------------------ --------------------- 5.1/11.1 MB 15.5 MB/s eta 0:00:01
  


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\akanksha.yadav\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedGroupKFold, cross_val_score


In [16]:
df=pd.read_csv("D:\\All Data till 21st Feb 2025\\D-MART ANALYSIS\\fd_tickets_202502181901.csv")
df.shape
df.isnull().sum()
df.fillna(df.mode().iloc[0],inplace=True)
df.isnull().sum()

Chain                    0
Store                    0
Store Address            0
State                    0
City                     0
Pin Code                 0
Terminal Category        0
Serial Number            0
Software Version         0
cf_utid                  0
First Trx Date           0
Last Trx Date            0
ticket_id                0
created_at               0
cf_issue_sub_category    0
issue_type               0
status                   0
dtype: int64

In [17]:
df.head()

,Chain,Store,Store Address,State,City,Pin Code,Terminal Category,Serial Number,Software Version,cf_utid,First Trx Date,Last Trx Date,ticket_id,created_at,cf_issue_sub_category,issue_type,status
0,DMART KARNATAKA,5616 D MART RAMMURTHY NGR,DMART AVENUE SUPERMARTS LTD ANR COMPLEX-RAMAMU...,KARNATAKA,BENGALURU,560016.0,Move 2500 4G,210867303061211220214180,M13_920810:uniPAY NEXT 8.0#02000000#WinXP_2.8....,M0015503,1/20/2022,2/17/2025,2983815,19/11/2024,Terminal Replacement - Working,Hardware Issue,resolved
1,DMART KARNATAKA,5616 D MART RAMMURTHY NGR,DMART AVENUE SUPERMARTS LTD ANR COMPLEX-RAMAMU...,KARNATAKA,BENGALURU,560016.0,Move 2500 4G,210867303061211220214180,M13_920810:uniPAY NEXT 8.0#02000000#WinXP_2.8....,M0015503,1/20/2022,2/17/2025,2983804,19/11/2024,Terminal Replacement - Working,Hardware Issue,resolved
2,DMART KARNATAKA,5616 D MART RAMMURTHY NGR,DMART AVENUE SUPERMARTS LTD ANR COMPLEX-RAMAMU...,KARNATAKA,BENGALURU,560016.0,Move 2500 4G,210867303061211220220672,M13_3829CF:uniPAY NEXT 8.0#02000000#WinXP_2.8....,M0015506,1/19/2022,2/19/2025,2983817,19/11/2024,Terminal Replacement - Working,Hardware Issue,resolved
3,DMART KARNATAKA,5616 D MART RAMMURTHY NGR,DMART AVENUE SUPERMARTS LTD ANR COMPLEX-RAMAMU...,KARNATAKA,BENGALURU,560016.0,Move 2500 4G,210867303061211220220672,M13_3829CF:uniPAY NEXT 8.0#02000000#WinXP_2.8....,M0015506,1/19/2022,2/19/2025,2983803,19/11/2024,Terminal Replacement - Working,Hardware Issue,resolved
4,DMART KARNATAKA,5631 DMART SRIGANDHA BANGALORE,DMART 6 A3 KISHAN ENTERPRISES HOYSALA-NAGAR NE...,KARNATAKA,BENGALURU,560091.0,Move 2500 4G,220637303061211223929476,M13_920810:uniPAY NEXT 8.0#02000000#WinXP_2.8....,M0016509,1/22/2022,2/19/2025,3002529,30/11/2024,Retrieval Post Replacement,Hardware Issue,open


In [18]:
# Convert date columns
date_columns = ["created_at", "First Trx Date", "Last Trx Date"]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors="coerce")
# Extract features
df["created_year"] = df["created_at"].dt.year
df["created_month"] = df["created_at"].dt.month
df["created_day"] = df["created_at"].dt.day
df["first_trx_year"] = df["First Trx Date"].dt.year
df["first_trx_month"] = df["First Trx Date"].dt.month
df["first_trx_day"] = df["First Trx Date"].dt.day
df["last_trx_year"] = df["Last Trx Date"].dt.year
df["last_trx_month"] = df["Last Trx Date"].dt.month
df["last_trx_day"] = df["Last Trx Date"].dt.day
# Drop original date columns


C:\Users\akanksha.yadav\AppData\Local\Temp\ipykernel_179440\3565829502.py:4: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors="coerce")


In [27]:
from sklearn.preprocessing import LabelEncoder

categorical_cols=["Chain","Store","State","City","Terminal Category","cf_issue_sub_category","issue_type","status"]

le=LabelEncoder()
for col in categorical_cols:
    df[col]=le.fit_transform(df[col])
df.columns=df.columns.str.strip()    

In [28]:
# Define target variable
y = df["Store"]
# Drop unnecessary columns
X = df.drop(["Store", "Store Address", "Serial Number", "Software Version", "cf_utid"], axis=1, errors="ignore")
# Check shape
print("Features Shape:", X.shape)
print("Target Shape:", y.shape)

from sklearn.model_selection import train_test_split
# Split into train (80%) and test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print("Training Set Shape:", X_train.shape)
print("Testing Set Shape:", X_test.shape)






Features Shape: (115504, 18)
Target Shape: (115504,)
Training Set Shape: (92403, 18)
Testing Set Shape: (23101, 18)


In [29]:
print(X_train.dtypes)


Chain                      int64
State                      int64
City                       int64
Pin Code                 float64
Terminal Category          int64
ticket_id                  int64
cf_issue_sub_category      int64
issue_type                 int64
status                     int64
created_year               int32
created_month              int32
created_day                int32
first_trx_year             int32
first_trx_month            int32
first_trx_day              int32
last_trx_year              int32
last_trx_month             int32
last_trx_day               int32
dtype: object


In [30]:
from imblearn.over_sampling import SMOTE
# Apply SMOTE to balance the dataset
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print("After SMOTE, Training Set Shape:", X_train.shape)


After SMOTE, Training Set Shape: (546800, 18)


In [31]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
# Define XGBoost model
xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    objective="multi:softmax",  # Multi-class classification
    num_class=len(y.unique()),
    random_state=42
)
# Train the model
xgb_model.fit(X_train, y_train)
# Make predictions
y_pred = xgb_model.predict(X_test)
# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"XGBoost Model Accuracy: {accuracy:.4f}")




: 

In [ ]:
xgb_model_cv = XGBClassifier(
    tree_method="gpu_hist",  # Use GPU for faster computation
    predictor="gpu_predictor",
    n_estimators=200,
    max_depth=6,
    learning_rate=0.05,
    objective="multi:softmax",
    num_class=len(y_train.unique()),
    random_state=42
)


